In [1]:
%pwd

'd:\\Advanced Project\\Lesion-Stroke-Segmentation\\notebooks'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'd:\\Advanced Project\\Lesion-Stroke-Segmentation'

In [21]:
lst_file = os.listdir(r"artifact\model_training\trained_model")
lst_file

['model-001-0.8611.keras', 'model-010-0.89.keras']

In [22]:
split_lst = []

In [23]:
for file in lst_file:
    if file.endswith('.keras'):
        split_file = file.split("-")
        val_loss = float(split_file[-1].replace(".keras", ""))
        split_lst.append([file, val_loss])

In [26]:
min(split_lst, key= lambda x: x[1])[0]

'model-001-0.8611.keras'

In [8]:
split_file[-1].replace(".keras", "")

'0.8611'

In [4]:
from pathlib import Path
from dataclasses import dataclass

In [5]:
@dataclass(frozen=True)
class ModelTrainingEntity:
    train_input_data: Path
    train_mask_data: Path
    trained_model: Path
    tensorboard_logs: Path
    batch_size: int
    nfilters: int
    drop_out: float
    nclasses: int
    focal_alpha_loss: float
    focal_gamma_loss: float
    learning_rate: float
    input_height: int
    input_width: int
    input_channel: int
    epochs: int
    final_class_activation: str
    activation: str
    kernel_initializer: str

In [6]:
from lesionSeg.constant import *
from lesionSeg.Utils.common import read_yaml, create_directory

In [7]:
class ConfigurationManager:
    def __init__(self, params=PARAMS_FILE_PATH, config=CONFIG_FILE_PATH):
        self.params = read_yaml(params)
        self.config = read_yaml(config)

        create_directory([self.config.artifact_root])

    def model_training_config(self):
        config = self.config.model_training
        params = self.params.model_training

        create_directory([config.root_dir])

        training_entity = ModelTrainingEntity(
            train_input_data = Path(config.train_input_data),
            train_mask_data = Path(config.train_mask_data),
            trained_model= Path(config.trained_model),
            tensorboard_logs = Path(config.tensorboard_logs),
            batch_size = params.batch_size,
            nfilters = params.nfilters,
            drop_out = params.drop_out,
            nclasses = params.nclasses,
            focal_alpha_loss = params.focal_alpha_loss,
            focal_gamma_loss = params.focal_gamma_loss,
            learning_rate = params.learning_rate,
            input_height = params.input_height,
            input_width = params.input_width,
            input_channel = params.input_channel,
            final_class_activation = params.final_class_activation,
            activation = params.activation,
            kernel_initializer = params.kernel_initializer,
            epochs = params.epochs,
        )

        return training_entity

In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from lesionSeg.models.unet_2d import UNet2D
from lesionSeg.logging import logger
from lesionSeg.Utils.common import CustomDataGenerator
from lesionSeg.Exception.exception import CustomeException
from lesionSeg.models.loss import focal_tversky_loss
from lesionSeg.models.metrices import dice_coff
import os
import sys
from typing import Tuple
from pathlib import Path

import numpy as np
np.Inf = np.inf

In [ ]:
class ModelTraining:
    def __init__(self, args, config: ModelTrainingEntity):
        self.args = args
        self.config = config
        self.input_shape = (self.config.input_height, 
                          self.config.input_width, 
                          self.config.input_channel)
        self._validate_paths()

    def _validate_paths(self) -> None:
        """Validate existence of required directories"""
        
        required_paths = {
            'input_dir': self.config.train_input_data,
            'output_dir': self.config.train_mask_data,
            'tensorboard_logs': self.config.tensorboard_logs
        }
        
        if self.args.train_type == 'fine_tune':
            required_paths['model_path'] = self.args.model_path

        for name, path in required_paths.items():
            if not Path(path).exists():
                raise CustomeException(f"{name} path does not exist: {path}", sys)

    def _create_data_generators(self) -> Tuple[tf.keras.utils.Sequence, tf.keras.utils.Sequence]:
        """Create training and validation data generators"""
        
        input_dir = self.config.train_input_data
        output_dir = self.config.train_mask_data
        # Get sorted file lists with validation
        input_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir)])
        output_files = sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir)])

        # Verify file correspondence
        if len(input_files) != len(output_files):
            raise CustomeException("Mismatch between input and output file counts", sys)
            
        # Split data
        X_train, X_valid, y_train, y_valid = train_test_split(
            input_files, output_files, 
            test_size=0.1, 
            random_state=42
        )
        logger.info(f"Training Samples: {len(X_train)}, Validation Samples: {len(X_valid)}")

        # Create generators
        train_gen = CustomDataGenerator(
            input_lst=X_train,
            output_lst=y_train,
            batch_size=self.config.batch_size,
            shuffle=True,
            augmentation=True
        )
        
        valid_gen = CustomDataGenerator(
            input_lst=X_valid,
            output_lst=y_valid,
            batch_size=self.config.batch_size,
            shuffle=False,
            augmentation=False
        )
        
        return train_gen, valid_gen

    def _get_model_callbacks(self) -> list:
        """Create and return training callbacks"""
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            f'best_model.keras',
            monitor="val_iou",
            mode="max",
            save_best_only=True,
            verbose=1,
            save_weights_only=False,
        )

        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor="val_iou",
            patience=15,
            mode="max",
            restore_best_weights=True
        )

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )

        tensorboard = tf.keras.callbacks.TensorBoard(
            log_dir=self.config.tensorboard_logs
        )

        return [checkpoint, early_stopping, reduce_lr, tensorboard]

    def _initialize_model(self, mode = 'new') -> tf.keras.models:
        """Initialize or load model based on training type"""
        if self.args.train_type == 'new':
            logger.info("Initializing new model")
            model = UNet2D(
                nfilters=self.config.nfilters,
                nclassess=self.config.nclasses,
                final_class_activation=self.config.final_class_activation,
                activation=self.config.activation,
                kernel_initializer=self.config.kernel_initializer,
                input_size=self.input_shape
            )
            unet = model.unet_model()
            return unet
            
        elif self.args.train_type == 'fine_tune':
            logger.info(f"Loading pre-trained model from {self.args.model_path}")
            try:
                return tf.keras.models.load_model(
                    self.args.model_path,
                    custom_objects={
                        'focal_tversky_loss': focal_tversky_loss,
                        'dice_coff': dice_coff,
                        'iou':tf.keras.metrics.IoU,
                        'precision':tf.keras.metrics.Precision,
                        'recall':tf.keras.metrics.Recall
                    }
                )
            except Exception as e:
                raise CustomeException(f"Error loading model: {str(e)}", sys) 

    def train_model(self) -> None:
        """Main training workflow"""
        
        # Create data generators
        train_gen, valid_gen = self._create_data_generators()
        
        # Initialize model
        model = self._initialize_model()
        # Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.learning_rate),
            loss=focal_tversky_loss,
            metrics=[
                dice_coff,
                tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1], name='iou'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')
            ]
        )
        
        # Train model
        history = model.fit(
            train_gen,
            epochs=self.config.epochs,
            validation_data=valid_gen,
            callbacks=self._get_model_callbacks(),
            verbose=1
        )

        model.save(self.config.trained_model)
        
        logger.info("Training completed successfully")

In [ ]:
try:
    config = ConfigurationManager()
    training_config = config.model_training_config()
    model_training = ModelTraining(training_config)
    model_training.train_model()
except Exception as e:
    raise e

[2025-02-19 23:34:42,534]: INFO: common : Read YAML File: params.yaml
[2025-02-19 23:34:42,536]: INFO: common : Read YAML File: config\config.yaml
[2025-02-19 23:34:42,539]: INFO: common : Directory has been Created: artifact
[2025-02-19 23:34:42,540]: INFO: common : Directory has been Created: model_training
[2025-02-19 23:34:42,643]: INFO: 1121927732 : Training Samples: 12379, Validation Samples: 1376
[2025-02-19 23:34:42,653]: INFO: 1121927732 : Initializing new model


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=best_model.h5

In [19]:
model = tf.keras.models.load_model('best_model.keras', custom_objects={
                        'focal_tversky_loss': focal_tversky_loss,
                        'dice_coff': dice_coff
                    })

d:\Advanced Project\Lesion-Stroke-Segmentation\env\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 166 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │      9,248 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout2d   │ (None, 32, 32,    │          0 │ max_pooling2d_1[… │
│ (SpatialDropout2D)  │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │     73,856 │ spatial_dropout2… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ batch_normalizat

 Total params: 23,303,845 (88.90 MB)

 Trainable params: 7,765,985 (29.62 MB)

 Non-trainable params: 5,888 (23.00 KB)

 Optimizer params: 15,531,972 (59.25 MB)

In [18]:
model.save('best_model.keras')